In [1]:
from utils.nlu_engine import NLUEngine
from utils.nlu_engine import DataUtils
from utils.nlu_engine import IntentMatcher, LR
from utils.nlu_engine import EntityExtractor
import nltk

# Example of intent and entity classification with NLU engine class
This is just a small example notebook to help users understand how to use the NLU engine.

* Intent example
* Entity example

Load data set. For this example, we will use the cleaned dataset, although you can load any dataset you like.

In [2]:
nlu_data_df = DataUtils.load_data(
    'data/NLU-Data-Home-Domain-Annotated-All-Cleaned.csv'
)

## Intent classification: example of a single utterance

Both the intents and the domains (scenarios/skills) can be used to label an utterance. In this example we will use domains to label the utterances' intents. 

In [ ]:
domains = nlu_data_df.scenario.values

LR_domain_classifier_model, tfidf_vectorizer = NLUEngine.train_intent_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict='domain',
    classifier=LR
)


In [ ]:
intent = nlu_data_df.intent.values

LR_domain_classifier_model, tfidf_vectorizer = NLUEngine.train_intent_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict='intent',
    classifier=LR
)


Example: Let's try to predict an utterances intent label using the domains.

In [ ]:
utterance = "turn off the kitchen lights"

print(IntentMatcher.predict_label(
    LR_domain_classifier_model, tfidf_vectorizer, utterance))

## Entity extraction

The entity extraction could be greatly improved by improving the features it uses. It would be great if someone would take a look at this. Perhaps the CRF features similar to what Snips uses would be better such as Brown clustering (probably).

It is important to have the NLTK tokenizer to be able to extract entities.

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

### Example: Extracting entities from an utterance

In [ ]:
crf_model = NLUEngine.train_entity_classifier(data_df=nlu_data_df)

In [ ]:
utterance = 'wake me up at five pm this week'

We can get the entity tags of a specific utterance with the EntityExtractor.


In [ ]:
EntityExtractor.get_entity_tags(utterance, crf_model)

We can also get the entity tagged utterance with the NLUEngine.


In [ ]:
entity_tagged_utterance = NLUEngine.create_entity_tagged_utterance(
    utterance, crf_model)

entity_tagged_utterance

In [ ]:
#TODO remove everything from here (perhaps move it into another notebook?), this was just to quickly evaluate entity matching using spaCy for PoS.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Get busy living or get busy dying.")

print(f"{'text':{8}} {'POS':{6}} {'TAG':{6}} {'Dep':{6}} {'POS explained':{20}} {'tag explained'} ")
for token in doc:
    print(f'{token.text:{8}} {token.pos_:{6}} {token.tag_:{6}} {token.dep_:{6}} {spacy.explain(token.pos_):{20}} {spacy.explain(token.tag_)}')

In [ ]:
list_of_words_and_tags = []
for token in doc:
    list_of_words_and_tags.append((token.text, token.tag_))

list_of_words_and_tags


In [ ]:
EntityExtractor.pos_tag_utterance(
    utterance="Get busy living or get busy dying.")


In [3]:
entity_reviewed_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=nlu_data_df)

Evaluating entity classifier


/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Cross validating with CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Time it took to cross validate CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100): 423.8120391368866


/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
entity_reviewed_report_df.to_csv('data/nltk_pos_entity_report.csv')

In [13]:
entity_reviewed_report_df


,entity-type,precision,recall,f1-score,support
0,0,0.879184,0.955116,0.915579,73501.000000
1,alarm_type,0.000000,0.000000,0.000000,30.000000
2,app_name,0.760870,0.460526,0.573770,76.000000
3,artist_name,0.485269,0.443740,0.463576,631.000000
4,audiobook_author,0.000000,0.000000,0.000000,29.000000
5,audiobook_name,0.000000,0.000000,0.000000,298.000000
6,business_name,0.374732,0.183054,0.245959,956.000000
7,business_type,0.714286,0.403226,0.515464,372.000000
8,change_amount,0.555556,0.209581,0.304348,167.000000
9,coffee_type,1.000000,0.183673,0.310345,49.000000


In [6]:
from utils.nlu_engine import Analytics
from utils.nlu_engine.entity_extractor import crf

In [10]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [8]:
def spacy_pos_tag_utterance(utterance):
    doc = nlp(utterance)
    list_of_words_and_tags = []
    for token in doc:
        list_of_words_and_tags.append((token.text, token.tag_))
    return list_of_words_and_tags

def create_feature_dataset(data_df):
    """
    Creates a feature dataset from the annotated utterances.
    """
    feature_dataset = []
    for utterance, utterance_with_tagging in zip(data_df['answer_normalised'], data_df['answer_annotation']):
        entities = EntityExtractor.extract_entities(utterance_with_tagging)
        utterance_pos = spacy_pos_tag_utterance(utterance)
        feature_dataset.append(
            EntityExtractor.combine_pos_and_entity_tags(entities, utterance_pos))
    return feature_dataset

def get_targets_and_labels(data_df):
    feature_dataset = create_feature_dataset(data_df)
    X = [EntityExtractor.utterance2features(utterance)
            for utterance in feature_dataset]
    y = [EntityExtractor.utterance2labels(utterance)
            for utterance in feature_dataset]

    return X, y

def evaluate_entity_classifier(data_df):
    """
    Evaluates the entity classifier and generates a report
    """

    print('Evaluating entity classifier')

    X, y = get_targets_and_labels(data_df)
    predictions = Analytics.cross_validate_classifier(crf, X, y)
    report_df = Analytics.generate_entity_classification_report(
        predictions, y)
    return report_df


In [11]:
entity_spacy_report_df = evaluate_entity_classifier(nlu_data_df)
entity_spacy_report_df

Evaluating entity classifier


/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Cross validating with CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Time it took to cross validate CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100): 402.08840775489807


/home/bartmoss/code/NLU-engine-prototype-benchmarks/.venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,entity-type,precision,recall,f1-score,support
0,0,0.883884,0.955293,0.918202,73412.000000
1,alarm_type,0.000000,0.000000,0.000000,30.000000
2,app_name,0.804878,0.434211,0.564103,76.000000
3,artist_name,0.456628,0.442155,0.449275,631.000000
4,audiobook_author,0.000000,0.000000,0.000000,29.000000
5,audiobook_name,0.000000,0.000000,0.000000,298.000000
6,business_name,0.403361,0.194332,0.262295,988.000000
7,business_type,0.689498,0.405914,0.510998,372.000000
8,change_amount,0.527778,0.227545,0.317992,167.000000
9,coffee_type,0.818182,0.183673,0.300000,49.000000


In [14]:
entity_spacy_report_df.to_csv('data/spacy_entity_report.csv')